In [75]:
import pandas as pd
import calendar
import numpy as np
import plotly.express as px
pd.options.plotting.backend = "plotly"

# Main data frame pre-processing

In [76]:
# Import main data frame
df = pd.read_csv('data/Melbourne_housing_FULL_OG.csv')

In [77]:
numToMonth = {
    "1": "January",
    "2": "February",
    "3": "March",
    "4": "April",
    "5": "May",
    "6": "June",
    "7": "July",
    "8": "August",
    "9": "September",
    "10": "October",
    "11": "November",
    "12": "December",
}

In [78]:
# df date splitting
df['Date'] = pd.to_datetime(df['Date'], format = '%d/%m/%Y')
df['Year'] = pd.DatetimeIndex(df['Date']).year
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Count'] = 1
df['MonthStr'] = df['Month'].apply(lambda x: numToMonth[str(x)])
df = df.drop(columns=['Date', 'Bedroom2', 'YearBuilt'])

In [79]:
# df['Postcode'] = df['Postcode'].astype(str)
df['Postcode'] = df['Postcode'].astype('Int64')
df['Bathroom'] = df['Bathroom'].astype('Int64')
df['Propertycount'] = df['Propertycount'].astype('Int64')
df['Landsize'] = df['Landsize'].astype('Int64')
df['Car'] = df['Car'].astype('Int64')

In [80]:
df.sample(10)

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Distance,Postcode,Bathroom,...,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Year,Month,Day,Count,MonthStr
31503,Blackburn,19 Patricia Rd,2,h,NaN,SN,Noel,13.4,3130,1,...,Whitehorse City Council,-37.81495,145.14177,Eastern Metropolitan,5713,2018,3,3,1,March
28732,Greensborough,1 Doris St,3,h,727000.0,SP,Fletchers,16.1,3088,1,...,Banyule City Council,-37.70760,145.09140,Northern Metropolitan,8524,2017,11,25,1,November
31275,Mulgrave,29 Shaftsbury Dr,3,h,828000.0,S,OBrien,18.8,3170,1,...,Monash City Council,-37.92096,145.16791,South-Eastern Metropolitan,7113,2018,2,10,1,February
4215,Fitzroy,124 Westgarth St,3,h,1481000.0,S,Jellis,1.6,3065,2,...,Yarra City Council,-37.79500,144.98060,Northern Metropolitan,5825,2016,8,13,1,August
452,Armadale,7/22 Derby St,1,u,450000.0,PI,hockingstuart,6.3,3143,1,...,Stonnington City Council,-37.85090,145.02030,Southern Metropolitan,4836,2016,9,24,1,September
2278,Brunswick,2/18 De Carle St,2,t,754000.0,S,Nelson,5.2,3056,2,...,Moreland City Council,-37.76250,144.96530,Northern Metropolitan,11918,2016,9,24,1,September
32514,Burwood,5/63 Somers St,2,u,500000.0,PI,Harcourts,10.4,3125,<NA>,...,Monash City Council,NaN,NaN,Southern Metropolitan,5678,2018,2,17,1,February
18530,Frankston,50 Towerhill Rd,4,h,755000.0,S,Ray,38.0,3199,1,...,Frankston City Council,-38.16146,145.12997,South-Eastern Metropolitan,17055,2017,7,15,1,July
22965,Black Rock,6 Prospect Gr,4,h,NaN,S,Buxton,17.3,3193,2,...,Bayside City Council,-37.97504,145.02062,Southern Metropolitan,2866,2017,8,26,1,August
32552,Craigieburn,9 Cascade Tce,4,h,662000.0,S,Ray,20.6,3064,2,...,Hume City Council,-37.58903,144.90065,Northern Metropolitan,15510,2018,2,17,1,February


In [81]:
df.dtypes

Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Distance         float64
Postcode           Int64
Bathroom           Int64
Car                Int64
Landsize           Int64
BuildingArea     float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount      Int64
Year               int64
Month              int64
Day                int64
Count              int64
MonthStr          object
dtype: object

In [82]:
missing_val_count_by_column = (df.isnull().sum())
missing_val_count_by_column

Suburb               0
Address              0
Rooms                0
Type                 0
Price             7610
Method               0
SellerG              0
Distance             1
Postcode             1
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21115
CouncilArea          3
Lattitude         7976
Longtitude        7976
Regionname           3
Propertycount        3
Year                 0
Month                0
Day                  0
Count                0
MonthStr             0
dtype: int64

In [83]:
df = df.drop(22632)
df = df.drop(29483)

# Creating seasonality line chart

In [84]:
sea = df.groupby(by=["Year","MonthStr"]).size().reset_index()
sea['Count'] = sea[0]
sea = sea.drop(columns=[0])
# sea[(sea['Year'] == year) & (sea['MonthStr'].isin(['September','October','November' ]))]['Count'].sum()
sea

,Year,MonthStr,Count
0,2016,April,502
1,2016,August,1173
2,2016,December,1000
3,2016,February,44
4,2016,January,3
5,2016,July,689
6,2016,June,1242
7,2016,May,1531
8,2016,November,1787
9,2016,October,852


# Creating count by suburbs

In [85]:
# Create count by suburb df
cbs = df.groupby(by=["Suburb", "Year"]).size().reset_index()
cbs['Count'] = cbs[0]
cbs = cbs.drop(columns=[0])

In [86]:
# Find total count for the year
yearTotal = cbs.groupby(by=["Year"]).size().reset_index()
yearTotal['Total'] = yearTotal[0]
yearTotal = yearTotal.drop(columns=[0])
yearTotal

,Year,Total
0,2016,142
1,2017,346
2,2018,311


In [87]:
# Find out ranks of each suburb
cbs = cbs.merge(yearTotal, on = 'Year', how = 'left')
cbs['Rank'] = cbs.groupby('Year')['Count'].rank(ascending=False,method='first').astype(int)
cbsTop = cbs.sort_values(by=['Rank']).head(30)

In [88]:
# Add corrections for cbsTop df
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Craigieburn') &  (cbs['Year'] == 2017)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Richmond') &  (cbs['Year'] == 2018)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Glen Iris') &  (cbs['Year'] == 2018)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Brighton') &  (cbs['Year'] == 2018)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Kew') &  (cbs['Year'] == 2018)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Balwyn North') &  (cbs['Year'] == 2017)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Preston') &  (cbs['Year'] == 2016)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Northcote') &  (cbs['Year'] == 2017)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Brunswick') &  (cbs['Year'] == 2016)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Camberwell') &  (cbs['Year'] == 2017)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Mill Park') &  (cbs['Year'] == 2017)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Hawthorn') &  (cbs['Year'] == 2017)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Glenroy') &  (cbs['Year'] == 2016)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Glenroy') &  (cbs['Year'] == 2018)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Brighton East') &  (cbs['Year'] == 2017)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'Mount Waverley') &  (cbs['Year'] == 2017)])
cbsTop = cbsTop.append(cbs.loc[(cbs['Suburb'] == 'South Yarra') &  (cbs['Year'] == 2018)])
cbsTop['Rank'] = cbsTop['Rank'].apply(lambda x: 11 if x > 10 else x)
cbsTop['Rank'] = cbsTop['Rank'].astype(str)
cbsTop['Rank'] = cbsTop['Rank'].apply(lambda x: "10+" if x == "11" else x)

In [89]:
cbsTop

,Suburb,Year,Count,Total,Rank
609,Reservoir,2018,90,311,1
607,Reservoir,2016,349,142,1
608,Reservoir,2017,405,346,1
77,Bentleigh East,2016,236,142,2
78,Bentleigh East,2017,288,346,2
216,Craigieburn,2018,75,311,2
611,Richmond,2017,265,346,3
79,Bentleigh East,2018,59,311,3
610,Richmond,2016,236,142,3
344,Glen Iris,2016,205,142,4


In [90]:
def findPostcode(suburbName):
    return(df.loc[df['Suburb'] == suburbName]['Postcode'].unique()[0])


In [91]:
cbsTop['Postcode'] = cbsTop['Suburb'].apply(lambda x: findPostcode(x))

In [92]:
cbsTop

,Suburb,Year,Count,Total,Rank,Postcode
609,Reservoir,2018,90,311,1,3073
607,Reservoir,2016,349,142,1,3073
608,Reservoir,2017,405,346,1,3073
77,Bentleigh East,2016,236,142,2,3165
78,Bentleigh East,2017,288,346,2,3165
216,Craigieburn,2018,75,311,2,3064
611,Richmond,2017,265,346,3,3121
79,Bentleigh East,2018,59,311,3,3165
610,Richmond,2016,236,142,3,3121
344,Glen Iris,2016,205,142,4,3146


# Examples

In [93]:
df[df['Method'] == 'VB']

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Distance,Postcode,Bathroom,...,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Year,Month,Day,Count,MonthStr
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,2.5,3067,2,...,Yarra City Council,-37.81140,145.01160,Northern Metropolitan,4019,2016,2,4,1,February
6,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,2.5,3067,1,...,Yarra City Council,-37.80720,144.99410,Northern Metropolitan,4019,2016,6,4,1,June
23,Abbotsford,411/8 Grosvenor St,2,u,700000.0,VB,Jellis,2.5,3067,2,...,Yarra City Council,-37.81100,145.00670,Northern Metropolitan,4019,2016,11,12,1,November
24,Abbotsford,40 Nicholson St,3,h,1350000.0,VB,Nelson,2.5,3067,2,...,Yarra City Council,-37.80850,144.99640,Northern Metropolitan,4019,2016,11,12,1,November
67,Airport West,50 Bedford St,3,h,730000.0,VB,Nelson,13.5,3042,2,...,Moonee Valley City Council,-37.72030,144.87550,Western Metropolitan,3464,2016,12,3,1,December
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34789,Thornbury,98 Hutton St,3,h,1200000.0,VB,Nelson,7.0,3071,1,...,Darebin City Council,-37.75468,144.98959,Northern Metropolitan,8870,2018,2,24,1,February
34802,Vermont,19 Caldwell Rd,4,h,900000.0,VB,Jellis,17.2,3133,1,...,Maroondah City Council,-37.83473,145.21116,Eastern Metropolitan,4181,2018,2,24,1,February
34817,Watsonia,32 Kenmare St,3,h,840000.0,VB,Stockdale,14.5,3087,1,...,Banyule City Council,-37.71152,145.07794,Northern Metropolitan,2329,2018,2,24,1,February
34840,Williamstown,3/2 Thompson St,2,u,520000.0,VB,Raine,6.8,3016,<NA>,...,Hobsons Bay City Council,NaN,NaN,Western Metropolitan,6380,2018,2,24,1,February


In [94]:
np.unique(df['Method'].values)

array(['PI', 'PN', 'S', 'SA', 'SN', 'SP', 'SS', 'VB', 'W'], dtype=object)

# Creating average values per suburb data frame

In [95]:
msd = df.groupby(by=['Suburb']).mean()

In [96]:
msd = msd[['Price', 'Rooms', 'Distance', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'Propertycount', 'Postcode']]

In [97]:
msd['Postcode'] = msd['Postcode'].astype(int).astype(str)
decimals = pd.Series([1, 1, 1, 1,1,1,1,0], index = ['Price', 'Rooms', 'Distance', 'Bathroom', 'Car', 'Landsize','BuildingArea', 'Propertycount'])
msd = msd.round(decimals)
msd

,Price,Rooms,Distance,Bathroom,Car,Landsize,BuildingArea,Propertycount,Postcode
Suburb,,,,,,,,,
Abbotsford,1033549.0,2.5,2.7,1.4,1.0,354.3,103.6,4019.0,3067
Aberfeldie,1307192.9,3.2,8.3,1.9,1.9,536.9,185.2,1543.0,3040
Airport West,751364.2,3.0,11.8,1.5,1.6,453.2,137.2,3464.0,3042
Albanvale,536055.6,3.1,14.0,1.5,2.2,527.8,127.4,1899.0,3021
Albert Park,1927650.5,2.9,3.2,1.6,0.8,198.2,136.9,3280.0,3206
...,...,...,...,...,...,...,...,...,...
Yallambie,820861.1,3.4,14.3,1.8,1.7,591.5,161.3,1369.0,3085
Yarra Glen,620000.0,3.0,31.4,2.0,1.0,863.0,180.6,1160.0,3775
Yarraville,991245.0,2.9,6.7,1.4,1.5,325.6,127.9,6543.0,3013


# Dot map data preparation

In [98]:
dm = df[["Suburb", "Address", "Rooms", "Type", "Price", "Distance", "Postcode", "Bathroom", "Car", "Landsize", "CouncilArea", "Lattitude","Longtitude","Regionname"]]

In [99]:
dm = dm.dropna(subset=["Price","Bathroom", "Car", "Lattitude", "Longtitude", "Landsize"])

In [100]:
dm.head()

,Suburb,Address,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,CouncilArea,Lattitude,Longtitude,Regionname
1,Abbotsford,85 Turner St,2,h,1480000.0,2.5,3067,1,1,202,Yarra City Council,-37.7996,144.9984,Northern Metropolitan
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,2.5,3067,1,0,156,Yarra City Council,-37.8079,144.9934,Northern Metropolitan
4,Abbotsford,5 Charles St,3,h,1465000.0,2.5,3067,2,0,134,Yarra City Council,-37.8093,144.9944,Northern Metropolitan
5,Abbotsford,40 Federation La,3,h,850000.0,2.5,3067,2,1,94,Yarra City Council,-37.7969,144.9969,Northern Metropolitan
6,Abbotsford,55a Park St,4,h,1600000.0,2.5,3067,1,2,120,Yarra City Council,-37.8072,144.9941,Northern Metropolitan


In [101]:
# Exclude for now
dm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17678 entries, 1 to 34856
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Suburb       17678 non-null  object 
 1   Address      17678 non-null  object 
 2   Rooms        17678 non-null  int64  
 3   Type         17678 non-null  object 
 4   Price        17678 non-null  float64
 5   Distance     17678 non-null  float64
 6   Postcode     17678 non-null  Int64  
 7   Bathroom     17678 non-null  Int64  
 8   Car          17678 non-null  Int64  
 9   Landsize     17678 non-null  Int64  
 10  CouncilArea  17678 non-null  object 
 11  Lattitude    17678 non-null  float64
 12  Longtitude   17678 non-null  float64
 13  Regionname   17678 non-null  object 
dtypes: Int64(4), float64(4), int64(1), object(5)
memory usage: 2.1+ MB


In [102]:
dm.isnull().sum()

Suburb         0
Address        0
Rooms          0
Type           0
Price          0
Distance       0
Postcode       0
Bathroom       0
Car            0
Landsize       0
CouncilArea    0
Lattitude      0
Longtitude     0
Regionname     0
dtype: int64

In [103]:
factor = "Bathroom"
print(dm[factor].min(), dm[factor].max())

0 9


In [104]:
dm['Type'].unique()

array(['h', 'u', 't'], dtype=object)

# Distribution of selling method and house types

In [105]:
fig = px.pie(df, values='Count', names='Method')
# fig.show()

In [106]:
method = 'PI'
df.loc[df['Method'] == method]['Price'].isnull().sum() / df.loc[df['Method'] == method]['Price'].size

0.3288659793814433

In [107]:
slm = df.groupby(by=["Method"]).size().reset_index()
slm['Count'] = slm[0]
slm = slm.drop(columns=[0])
# Maybe add a description
totalCount = slm['Count'].sum()
slm['ratio_of_total'] = slm['Count']/totalCount
slm['Description'] = pd.Series(dtype='object')
fullname = ['Property passed in', 'Sold prior not disclosed', 'Property sold', 'Sold after auction', 'Sold not disclosed', 'Property sold prior', 'Sold after auction, price not disclosed', 'Vendor bid', 'Withdrawn prior to auction']
slm['Name'] = fullname

In [108]:
slm.at[0, 'Description'] = 'Property passed in - occurs when the top bid is below the reserve price meaning the property doesn''t sell through an auction process. From there, the highest bidder can negotiate with the seller. '
slm

,Method,Count,ratio_of_total,Description,Name
0,PI,4850,0.139148,Property passed in - occurs when the top bid i...,Property passed in
1,PN,308,0.008837,NaN,Sold prior not disclosed
2,S,19743,0.566432,NaN,Property sold
3,SA,226,0.006484,NaN,Sold after auction
4,SN,1317,0.037785,NaN,Sold not disclosed
5,SP,5094,0.146148,NaN,Property sold prior
6,SS,36,0.001033,NaN,"Sold after auction, price not disclosed"
7,VB,3108,0.089169,NaN,Vendor bid
8,W,173,0.004963,NaN,Withdrawn prior to auction


In [109]:
totalCount

34855

In [110]:
fig = px.pie(df, values='Count', names='Type')
# fig.show()

# Export & Clean-up

In [111]:
# Export to CSV after preprocessing
df.to_csv('data/Melbourne_housing_FULL.csv')
cbsTop.to_csv('data/countsBySuburb.csv')
msd.to_csv('data/meanSuburbValues.csv')
dm.to_csv('data/dotmapHousing.csv')
slm.to_csv('data/sellingMethodDist.csv')